# 🚀 ETL del dataset `output_steam_games`
#### En este notebook se desarrolla la 📦 extracción, 💱 transformación y 📥 carga del conjunto de datos `output_steam_games`

#### 📥Importaciones 

In [3]:
import pandas as pd
import json
import re 

#### 📦 Extracción de los datos y primera exploración

Se extraen los datos desde el archivo json, se convierte en Dataframe y se realiza una observación de su contenido.

In [4]:
# Ruta al dataset 
ruta_juegos = './output_steam_games.json'

# Se lee de cada línea del dataset
filas_juegos = []
with open(ruta_juegos, encoding='utf-8') as archivo:
    for line in archivo:
        data = json.loads(line)
        filas_juegos.append(data)

# Se convierte en dataframe
df_juegos = pd.DataFrame(filas_juegos)
df_juegos

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


#### 💱 Transformacion de los datos

Se observan las filas que contienen vacíos en todo el registro y se eliminan dichos registros.

In [5]:
df_juegos = df_juegos.dropna(how='all').reset_index(drop=True)
df_juegos.shape

(32135, 13)

Se revisan los tipos de datos por columna y la cantidad de nulos.

In [6]:
def verificar_tipo_datos(df):
    
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
            
    return df_info

In [7]:
verificar_tipo_datos(df_juegos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'float'>]",74.94,25.06,8052
1,genres,"[<class 'list'>, <class 'float'>]",89.78,10.22,3283
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38,2050
4,url,[<class 'str'>],100.00,0.00,0
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43,2067
6,tags,"[<class 'list'>, <class 'float'>]",99.49,0.51,163
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
8,specs,"[<class 'list'>, <class 'float'>]",97.92,2.08,670
9,price,"[<class 'float'>, <class 'str'>]",95.71,4.29,1377


Luego de borrar las filas NaN se observan 13 columnas y 32135 filas.

 Aún contienen nulos algunas columnas. 

 Se revisan algunas columnas para conocer su estructura.

In [8]:
# se observa el tipo de dato en 'genres'
df_juegos['genres'][0]

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

In [9]:
# se observa el tipo de dato en 'tags'
df_juegos['tags'][0]

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']

In [10]:
# se observa el tipo de dato en 'specs'
df_juegos['specs'][100]

['Single-player', 'Multi-player']

In [11]:
# se observa el tipo de dato en 'price'
df_juegos['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [12]:
# se observa el tipo de dato en 'early_access'
df_juegos['early_access'].unique()

array([False, True], dtype=object)

Las columnas que contiene este conjunto son:   

* **publisher**: es la empresa publicadora del contenido.
* **genres**: es el género del item, es decir, del juego. Esta formado por una lista de uno o mas géneros por registro.
* **app_name**: es el nombre del item, es decir, del juego.
* **title**: es el título del item.
* **url**: es la url del juego.
* **release_date**: es la fecha de lanzamiento del item en formato 0000-00-00.
* **tags**: es la etiqueta del contenido. Esta formado por una lista de uno o mas etiquetas por registro.
* **reviews_url**: es la url donde se encuentra el review de ese juego.
* **specs**: son especificaciones de cada item. Es una lista con uno o mas string con las especificaciones.
* **price**: es el precio del item.
* **early_access**: indica el acceso temprano con un True/False.
* **id**: es el identificador único del contenido.
* **developer**: es el desarrollador del contenido.

#### ✅​ Verificación de duplicados

Se analizan si hay duplicados teniendo en cuenta la columna del id del item.

In [13]:
def verifica_duplicados_por_columna(df, columna):
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"
    
    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

In [14]:
verifica_duplicados_por_columna(df_juegos, 'id')

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Se observa que solo hay dos items duplicados. 

Por un lado el 'id' 612880 que se observa es el mismo registro en los dos casos. Por lo que se puede eliminar cualquier de ellos. 

Por otro lado, se observa que la segunda dupla de duplicados el 'id' es NaN y la primer fila presenta mas cantidad de NaN que la segunda.



In [15]:
df_juegos[df_juegos['developer']=='Rocksteady Studios,Feral Interactive (Mac)']

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


Se puede observar que el item que no tiene 'id' es el 'id' 200260 y que el registro se encuentra completo. Por lo tanto, se pueden borrar ambas filas dupliacadas.

In [16]:
# se eliminan index duplicados
index_a_eliminar = [14573, 74, 30961]
df_juegos = df_juegos.drop(index_a_eliminar)

#### 💱 Transformación de la columna 'release_date'

Se observan las cantidades de registros por cada fecha

In [17]:
df_juegos['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

Se crea una funcion para reemplazar valores los nulos por 'Dato no disponible'

In [18]:
def obtener_anio_release(fecha):
    if pd.notna(fecha):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', fecha):
            return fecha.split('-')[0]
    return 'Dato no disponible'

Se crea una columna nueva con el año

In [19]:
df_juegos['release_anio'] = df_juegos['release_date'].apply(obtener_anio_release)

Se elimina la columna 'release_date'

In [20]:
df_juegos = df_juegos.drop('release_date', axis=1)
df_juegos.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_anio
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Dato no disponible


In [21]:
df_juegos['release_anio'].unique()

array(['2018', '2017', 'Dato no disponible', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

#### 💱 Transformación de las columnas 'publisher', 'app_name', 'title', 'developer'

En estas columnas se observan valores nulos, por lo que se decide completarlos con valores del tipo 'Dato no disponible' para que el tipo de dato sea uno solo en la columna.

In [22]:
# Columnas a transformar
columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']

# Se rellenan los nulos
df_relleno = df_juegos[columnas_a_completar].fillna('Dato no disponible')

# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
df_juegos = pd.concat([df_juegos.drop(columnas_a_completar, axis=1), df_relleno], axis=1)
df_juegos.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Dato no disponible,Log Challenge,Dato no disponible,Dato no disponible


#### 💱 Transformación columna 'genres'

La columna 'genres' esta formada por una lista de los distintos géneros de los videojuegos. Se necesita hacer una columna con cada género.

In [23]:
df_juegos = df_juegos.explode('genres')
df_juegos = df_juegos.dropna(subset=['genres'])
df_juegos.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


#### 💱 Transformación columnas 'tags', 'specs', 'url', 'reviews_url', 'price' y 'early_access'

Estas columnas no se utilizarán para las consultas de la API ni para el modelo de recomendación, por lo tanto se eliminan del dataframe.

In [24]:
df_juegos = df_juegos.drop(['tags', 'specs', 'url', 'reviews_url', 'price', 'early_access'], axis=1)
df_juegos.columns

Index(['genres', 'id', 'release_anio', 'publisher', 'app_name', 'title',
       'developer'],
      dtype='object')

#### ✅ Finalmente, se verifican el tipo de dato y si quedan nulos.

In [25]:
verificar_tipo_datos(df_juegos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,id,[<class 'str'>],100.0,0.0,0
2,release_anio,[<class 'str'>],100.0,0.0,0
3,publisher,[<class 'str'>],100.0,0.0,0
4,app_name,[<class 'str'>],100.0,0.0,0
5,title,[<class 'str'>],100.0,0.0,0
6,developer,[<class 'str'>],100.0,0.0,0


## 📥 Carga del dataset `output_steam_games.json`

Se guarda el dataframe transformado como `steam_games_limpio`.

In [26]:
archivo_limpio = 'Data/steam_games_limpio.csv'
df_juegos.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Data/steam_games_limpio.csv
